# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
%%capture
%pip install tensorflow_io sagemaker -U

In [2]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

Save the IAM role in a variable called `role`. This would be useful when training the model.

In [3]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::910340444796:role/service-role/AmazonSageMaker-ExecutionRole-20230404T010108


In [4]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
        'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://my-object-detection-project/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the the repository
* build the docker image and push it 
* print the container name

In [5]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Objecimage_name = 'tf2-object-detection'
!sh ./docker/build_and_push.sh $image_namet Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

fatal: destination path 'docker/models' already exists and is not an empty directory.
bash: line 6: !sh: command not found


In [6]:
# build and push the docker image. This code can be commented after being ran once.
# This will take around 10 mins.
#image_name = 'tf2-object-detection'
#!sh ./docker/build_and_push.sh $image_name

To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [7]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

910340444796.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20230404231810


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be ajusted if you were to experiment with other architectures.

In [8]:
%%bash
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint

ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.index


mkdir: cannot create directory ‘source_dir/checkpoint’: File exists
--2023-04-05 03:23:54--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.31.128, 2607:f8b0:4004:c17::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.31.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20518283 (20M) [application/x-tar]
Saving to: ‘/tmp/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 17.7M 1s
    50K .......... .......... .......... .......... ..........  0% 32.5M 1s
   100K .......... .......... .......... .......... ..........  0% 32.4M 1s
   150K .......... .......... .......... .......... ..........  0% 88.6M 1s
   200K .......... .......... .......... .......... ..........  1%  134M 1s
   250K .......... .......... .......... ....

## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [13]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir":"/opt/training",        
        "pipeline_config_path": "pipeline.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    #instance_type='ml.trn1.2xlarge',
    instance_type='ml.trn1.2xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)



INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [14]:
history = estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: tf2-object-detection-2023-04-05-03-47-01-765


2023-04-05 03:47:04 Starting - Starting the training job...
2023-04-05 03:47:18 Starting - Preparing the instances for training...
2023-04-05 03:48:06 Downloading - Downloading input data...
2023-04-05 03:48:26 Training - Downloading the training image............
2023-04-05 03:50:42 Training - Training image download completed. Training in progress.....2023-04-05 03:51:11,235 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-05 03:51:11,237 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-05 03:51:11,248 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-05 03:51:11,250 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-05 03:51:11,261 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-05 03:51:11,263 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neu

W0405 03:51:27.675414 140711772489536 module_wrapper.py:149] From /usr/local/lib/python3.8/dist-packages/object_detection/builders/optimizer_builder.py:124: The name tf.keras.optimizers.SGD is deprecated. Please use tf.keras.optimizers.legacy.SGD instead.
/usr/local/lib/python3.8/dist-packages/keras/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
I0405 03:51:32.472464 140707371132672 api.py:459] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
I0405 03:51:39.469773 140707371132672 api.py:459] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
Instructions for updating:
Use fn_output_signature instead
W0405 03:51:44.943434 140706406459136 deprecation.py:569] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/deprecat

INFO:tensorflow:Step 1100 per-step time 1.760s
I0405 04:23:00.610111 140711772489536 model_lib_v2.py:705] Step 1100 per-step time 1.760s
INFO:tensorflow:{'Loss/classification_loss': 0.24700218,
 'Loss/localization_loss': 0.30025256,
 'Loss/regularization_loss': 0.15098786,
 'Loss/total_loss': 0.6982426,
 'learning_rate': 0.07999918}
I0405 04:23:00.610412 140711772489536 model_lib_v2.py:708] {'Loss/classification_loss': 0.24700218,
 'Loss/localization_loss': 0.30025256,
 'Loss/regularization_loss': 0.15098786,
 'Loss/total_loss': 0.6982426,
 'learning_rate': 0.07999918}
INFO:tensorflow:Step 1200 per-step time 1.751s
I0405 04:25:55.730201 140711772489536 model_lib_v2.py:705] Step 1200 per-step time 1.751s
INFO:tensorflow:{'Loss/classification_loss': 0.1648966,
 'Loss/localization_loss': 0.24811147,
 'Loss/regularization_loss': 0.15086247,
 'Loss/total_loss': 0.56387055,
 'learning_rate': 0.079996705}
I0405 04:25:55.730523 140711772489536 model_lib_v2.py:708] {'Loss/classification_loss': 

W0405 04:49:31.693913 140241026598720 module_wrapper.py:149] From /usr/local/lib/python3.8/dist-packages/object_detection/builders/optimizer_builder.py:124: The name tf.keras.optimizers.SGD is deprecated. Please use tf.keras.optimizers.legacy.SGD instead.
INFO:tensorflow:Waiting for new checkpoint at /opt/training
I0405 04:49:31.694293 140241026598720 checkpoint_utils.py:168] Waiting for new checkpoint at /opt/training
INFO:tensorflow:Found new checkpoint at /opt/training/ckpt-3
I0405 04:49:31.694818 140241026598720 checkpoint_utils.py:177] Found new checkpoint at /opt/training/ckpt-3
/usr/local/lib/python3.8/dist-packages/keras/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
I0405 04:49:36.623837 140241026598720 api.py:459] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20),

I0405 04:54:48.736579 140610458367808 api.py:459] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
I0405 04:54:56.862602 140610458367808 api.py:459] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
I0405 04:54:59.868999 140610458367808 api.py:459] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
W0405 04:55:01.717837 140610458367808 save_impl.py:66] Skipping full serialization of Keras layer <object_detection.meta_architectures.ssd_meta_arch.SSDMetaArch object at 0x7fe1e89553d0>, because it is not built.
W0405 04:55:01.935618 140610458367808 save_impl.py:66] Skipping full serialization of Keras layer <keras.layers.convolutional.separable_conv2d.SeparableConv2D object at 0x7fe1cc242ca0>, because it is not built.
W0405 04:55:01.935810 140610458367808 save_impl.py:66] Skipping full serialization of Keras layer <object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fe1cc0f4ee0>, because it is

W0405 04:55:15.674380 140610458367808 save.py:274] Found untraced functions such as WeightSharedConvolutionalBoxPredictor_layer_call_fn, WeightSharedConvolutionalBoxPredictor_layer_call_and_return_conditional_losses, WeightSharedConvolutionalBoxHead_layer_call_fn, WeightSharedConvolutionalBoxHead_layer_call_and_return_conditional_losses, WeightSharedConvolutionalClassHead_layer_call_fn while saving (showing 5 of 173). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: /tmp/exported/saved_model/assets
I0405 04:55:20.652599 140610458367808 builder_impl.py:797] Assets written to: /tmp/exported/saved_model/assets
INFO:tensorflow:Writing pipeline config file to /tmp/exported/pipeline.config
I0405 04:55:21.775639 140610458367808 config_util.py:253] Writing pipeline config file to /tmp/exported/pipeline.config
2023-04-05 04:55:23,233 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2023-04-05 04:55:39 Uploading - Uploading generated 

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the intial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the writeup.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your writeup), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).